In [2]:
%pip install openai graphrag pandas requests python-dotenv
from IPython.display import clear_output ; clear_output()

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

is_azure = (
  os.getenv("AZURE_OPENAI_ENDPOINT", default="") != "" and
  os.getenv("OPENAI_API_KEY", default="") == ""
)

if is_azure:
  AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
  AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
  GPT_4_O_MODEL_NAME = os.getenv("GPT_4_O_MODEL_NAME", default="gpt-4o")
  TEXT_EMBEDDING_3_LARGE_MODEL_NAME = os.getenv("TEXT_EMBEDDING_3_LARGE_MODEL_NAME", default="text-embedding-3-large")
  AZURE_OPENAI_API_VERSION = "2024-05-01-preview"
else:
  OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
import requests
import re
import os
import pandas as pd

if not os.path.exists('data'):
  os.makedirs('data')

if not os.path.exists('data/wiki_pages.parquet'):
  wiki_pages = pd.DataFrame(columns=['Page', 'Content'])
  french_revolution = requests.get("https://en.wikipedia.org/wiki/French_Revolution?action=raw").text
  wiki_pages.loc[0] = ['French_Revolution', french_revolution]

  wiki_links = re.findall(r'\[\[(.*?)\]\]', french_revolution)
  for link in wiki_links:
    link = link.replace(' ', '_').split('|')[0].split('#')[0]
    if link not in wiki_pages['Page'] and ':' not in link:
      url = f"https://en.wikipedia.org/wiki/{link}?action=raw"
      response = requests.get(url)
      if response.status_code == 200:
        if not response.text.startswith('#REDIRECT'):
          wiki_pages.loc[len(wiki_pages)] = [link, response.text]

  wiki_pages.to_parquet('data/wiki_pages.parquet')
else:
  wiki_pages = pd.read_parquet('data/wiki_pages.parquet')

wiki_pages

,Page,Content
0,French_Revolution,{{Short description|Revolution in France from ...
1,Atlantic_Revolutions,{{Short description|1765–1838 series of revolu...
2,Storming_of_the_Bastille,{{short description|Major event of the French ...
3,Kingdom_of_France,{{Short description|Kingdom in Western Europe ...
4,Proclamation_of_the_abolition_of_the_monarchy,{{short description|Replacement of the French ...
...,...,...
450,Bloomsbury_Publishing,{{Short description|British worldwide publishi...
451,Google_Books,{{short description|Service from Google}}\n{{R...
452,Oxford_University_Press,{{Short description|Publishing arm of the Univ...
453,Ancien_Régime_in_France,#REDIRECT [[Ancien régime]]
